
# 📘 High Garden Coffee — **Evaluación (Holdout)**

Este notebook carga **predicciones** existentes (si las hay) o construye **baselines** y genera un **Resumen Ejecutivo** con métricas, tablas útiles y archivos de salida listos para presentar.


In [ ]:

# === Setup básico (sin dependencias nuevas) ===
from __future__ import annotations
from pathlib import Path
import json, math, glob, textwrap
from datetime import datetime
import pandas as pd
import numpy as np

pd.set_option("display.float_format", lambda x: f"{x:,.6f}")
print("✅ Entorno listo — pandas:", pd.__version__)


In [ ]:

# === Configuración ===
# Cambia el TARGET si quieres evaluar otro objetivo: 'price' | 'consumption' | 'profit'
TARGET = "price"  # <-- puedes cambiar aquí
# Si conoces el holdout explícito, ponlo aquí; si no, el notebook lo detecta.
HOLDOUT_YEAR = None  # p.ej., 2020

# Rutas típicas del repo
DIR_PRED = Path("predicciones")
DIR_RES  = Path("results")
DIR_DATA = Path("data")
DIR_MODELS = Path("models")

DIR_RES.mkdir(parents=True, exist_ok=True)

print("TARGET:", TARGET)
print("HOLDOUT_YEAR:", HOLDOUT_YEAR)


In [ ]:

# === Utilidades ===
def print_header(title: str):
    bar = "—"*112
    print(bar)
    print(title.upper())
    print(bar)

def read_csv_safe(path: Path) -> pd.DataFrame | None:
    try:
        if path.exists():
            return pd.read_csv(path)
    except Exception as e:
        print(f"[WARN] No pude leer {path}: {e}")
    return None

def format_metrics(m: dict) -> str:
    keys = ["RMSE", "MAE", "sMAPE(%)", "R2", "n_val", "year_val"]
    out = []
    for k in keys:
        v = m.get(k, None)
        if isinstance(v, float):
            if "sMAPE" in k:
                out.append(f"{k:<10}: {v:,.4f}")
            else:
                out.append(f"{k:<10}: {v:,.6f}")
        elif v is None:
            continue
        else:
            out.append(f"{k:<10}: {v}")
    return "\n".join(out)

def mae(y, yhat): 
    y, yhat = np.asarray(y, float), np.asarray(yhat, float)
    return float(np.mean(np.abs(y - yhat)))

def rmse(y, yhat):
    y, yhat = np.asarray(y, float), np.asarray(yhat, float)
    return float(math.sqrt(np.mean((y - yhat)**2)))

def smape(y, yhat):
    y, yhat = np.asarray(y, float), np.asarray(yhat, float)
    denom = (np.abs(y) + np.abs(yhat))
    return float(np.mean(np.where(denom == 0, 0.0, 2.0*np.abs(y - yhat)/(denom+1e-12))) * 100)

def r2(y, yhat):
    y, yhat = np.asarray(y, float), np.asarray(yhat, float)
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    return float(1 - ss_res/(ss_tot + 1e-12))

def detect_columns(df: pd.DataFrame, target: str):
    # intenta detectar y_true y y_pred en un DF
    y_true_cands = [c for c in df.columns if c in ["y_true", target]]
    y_pred_cands = [c for c in df.columns if c in ["y_pred", f"pred_{target}"]]
    y_true = y_true_cands[0] if y_true_cands else None
    y_pred = y_pred_cands[0] if y_pred_cands else None
    return y_true, y_pred

def load_master_df():
    # orden de preferencia
    for cand in [DIR_DATA/"coffee_clean.csv", Path("coffee_clean.csv"), Path("coffee_db.csv")]:
        dfm = read_csv_safe(cand)
        if dfm is not None:
            return dfm
    return None

def pick_latest(paths: list[str]) -> Path | None:
    paths = [Path(p) for p in paths if Path(p).exists()]
    if not paths:
        return None
    paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return paths[0]


## 1) Carga de predicciones (si existen)

In [ ]:

print_header("Búsqueda de archivos de predicción")
candidates = []
candidates += glob.glob(str(DIR_PRED / f"pred_{TARGET}*.csv"))
candidates += glob.glob(str(DIR_RES  / f"preds_{TARGET}.csv"))
cand = pick_latest(candidates)

if cand is None:
    print("No encontré archivos de predicción en predicciones/ ni results/.")
    PRED_PATH = None
    df_pred = None
else:
    PRED_PATH = Path(cand)
    print("Usando predicciones:", PRED_PATH)
    df_pred = read_csv_safe(PRED_PATH)

if df_pred is not None:
    print("Shape:", df_pred.shape)
    display(df_pred.head(3))


In [ ]:

print_header("Validación y enriquecimiento de predicciones")
y_true_col, y_pred_col = (None, None)
if df_pred is not None:
    y_true_col, y_pred_col = detect_columns(df_pred, TARGET)
    if y_true_col and y_pred_col:
        print("Columnas detectadas:", y_true_col, "|", y_pred_col)
    else:
        print("No encontré columnas de y_true/y_pred directamente. Intentaré reconstruir y_true desde data maestra…")
        # Intentar merge con master
        master = load_master_df()
        if master is not None and TARGET in master.columns:
            # claves comunes
            keys = [k for k in ["year","country","type"] if k in df_pred.columns and k in master.columns]
            if keys:
                df_pred = df_pred.merge(master[keys+[TARGET]], on=keys, how="left")
                y_true_col = TARGET
                if f"pred_{TARGET}" in df_pred.columns:
                    y_pred_col = f"pred_{TARGET}"
                elif "y_pred" in df_pred.columns:
                    y_pred_col = "y_pred"
                print("Reconstruidas columnas:", y_true_col, "|", y_pred_col, " (vía merge)")
            else:
                print("[WARN] No hay claves comunes para merge. No se pudo reconstruir y_true.")
        else:
            print("[WARN] No se pudo cargar data maestra o no contiene el target.")

# Filtro y limpieza final
if df_pred is not None and y_true_col and y_pred_col:
    cols_keep = [c for c in [y_true_col, y_pred_col, "year", "country", "type"] if c in df_pred.columns]
    df_eval = df_pred[cols_keep].copy()
    df_eval = df_eval.dropna()
    if df_eval.empty:
        print("[WARN] No hay filas evaluables tras dropna().")
    else:
        display(df_eval.head(5))
else:
    df_eval = None


In [ ]:

print_header("Métricas globales del modelo (si hay predicciones)")
metrics_model = None
if df_eval is not None and not df_eval.empty:
    y = df_eval[y_true_col].astype(float).to_numpy()
    yhat = df_eval[y_pred_col].astype(float).to_numpy()
    metrics_model = {
        "RMSE": rmse(y, yhat),
        "MAE": mae(y, yhat),
        "sMAPE(%)": smape(y, yhat),
        "R2": r2(y, yhat),
        "n_val": int(len(df_eval)),
    }
    # detectar holdout year si no viene seteado
    if HOLDOUT_YEAR is None and "year" in df_eval.columns:
        years = sorted(df_eval["year"].dropna().unique().tolist())
        HOLDOUT_YEAR = int(years[-1]) if years else None
    if HOLDOUT_YEAR is not None:
        metrics_model["year_val"] = HOLDOUT_YEAR
    
    print(format_metrics(metrics_model))
else:
    print("No hay predicciones evaluables. Continuaré con baselines y/o prepararé plantilla de evaluación.")


## 2) Comparación contra **baselines** (si hay data maestra)

In [ ]:

print_header("Construcción y evaluación de baselines")
baseline_metrics = {}
df_baselines_view = None

master = load_master_df()
if master is None or TARGET not in (master.columns if master is not None else []):
    print("No pude cargar data maestra o no contiene la columna target. Saltando baselines.")
else:
    # normalizar tipos
    if "year" in master.columns:
        master["year"] = pd.to_numeric(master["year"], errors="coerce").astype("Int64")
    # seleccionar holdout
    if HOLDOUT_YEAR is None and df_eval is not None and "year" in df_eval.columns and not df_eval.empty:
        years = sorted(df_eval["year"].dropna().unique().tolist())
        HOLDOUT_YEAR = int(years[-1]) if years else None
        print("Detecté HOLDOUT_YEAR:", HOLDOUT_YEAR)
    elif HOLDOUT_YEAR is None:
        # fallback: último año en master
        years_m = sorted(master["year"].dropna().unique().tolist()) if "year" in master.columns else []
        HOLDOUT_YEAR = int(years_m[-1]) if years_m else None
        print("Usaré HOLDOUT_YEAR (fallback):", HOLDOUT_YEAR)

    if HOLDOUT_YEAR is None:
        print("[WARN] No se pudo determinar el año de holdout. No puedo crear baselines temporales.")
    else:
        # Filtrar histórico y holdout
        keys = [k for k in ["country","type"] if k in master.columns]
        hist = master[master["year"] < HOLDOUT_YEAR].copy()
        test = master[master["year"] == HOLDOUT_YEAR].copy()
        if not keys:
            # si no hay llaves de grupo, trabajamos a nivel total
            hist["_grp"] = "all"
            test["_grp"] = "all"
            keys = ["_grp"]

        # Baseline 1: Último valor por grupo (LOCF)
        last_hist = hist.sort_values("year").groupby(keys)[TARGET].last().rename("yhat_last").reset_index()
        base_last = test.merge(last_hist, on=keys, how="left")

        # Baseline 2: Promedio histórico por grupo
        mean_hist = hist.groupby(keys)[TARGET].mean().rename("yhat_mean").reset_index()
        base_mean = test.merge(mean_hist, on=keys, how="left")

        # Si tenemos df_eval con y_true/y_pred, nos alineamos a sus llaves para comparación justa
        align_keys = [k for k in ["year"]+keys if df_eval is not None and k in df_eval.columns]
        if df_eval is not None and align_keys:
            gold = df_eval.rename(columns={y_true_col:"y_true"})[align_keys+["y_true"]].drop_duplicates()
        else:
            gold = test[["year"]+keys+[TARGET]].rename(columns={TARGET:"y_true"})

        dfb = gold.merge(base_last, on=["year"]+keys, how="left").merge(base_mean, on=["year"]+keys, how="left")
        # Métricas
        def met(df, col):
            ok = df[["y_true", col]].dropna()
            if ok.empty: return None
            return {
                "RMSE": rmse(ok["y_true"], ok[col]),
                "MAE": mae(ok["y_true"], ok[col]),
                "sMAPE(%)": smape(ok["y_true"], ok[col]),
                "n_val": int(len(ok)),
                "year_val": int(HOLDOUT_YEAR),
            }
        m_last = met(dfb, "yhat_last")
        m_mean = met(dfb, "yhat_mean")
        if m_last: 
            baseline_metrics["last_value"] = m_last
            print("Baseline — último valor:
" + format_metrics(m_last))
        if m_mean:
            baseline_metrics["mean_hist"] = m_mean
            print("\nBaseline — promedio histórico:
" + format_metrics(m_mean))

        # vista tabular
        show_cols = [c for c in ["year"]+keys+["y_true","yhat_last","yhat_mean"] if c in dfb.columns]
        df_baselines_view = dfb[show_cols].head(20)
        if not df_baselines_view.empty:
            print("\nMuestra de baselines (primeras 20 filas):")
            display(df_baselines_view)


## 3) Análisis de errores del modelo (si hay predicciones)

In [ ]:

print_header("Ranking de errores por observación y por país/tipo")
df_errors_rank = None
if df_eval is not None and not df_eval.empty:
    dfm = df_eval.copy()
    dfm["abs_err"] = (dfm[y_true_col] - dfm[y_pred_col]).abs()
    dfm["pct_err"] = np.where(dfm[y_true_col]!=0, dfm["abs_err"]/dfm[y_true_col]*100, np.nan)

    # Top 10 peores / mejores (por error absoluto)
    worst10 = dfm.sort_values("abs_err", ascending=False).head(10)
    best10  = dfm.sort_values("abs_err", ascending=True).head(10)

    print("🔴 Peores 10 (mayor error absoluto):")
    display(worst10)
    print("\n🟢 Mejores 10 (menor error absoluto):")
    display(best10)

    # Agregados por país y tipo (si existen)
    keys = [k for k in ["country","type"] if k in dfm.columns]
    if keys:
        agg = dfm.groupby(keys).agg(
            n=("abs_err","count"),
            MAE=("abs_err","mean"),
            RMSE=("abs_err", lambda s: float(math.sqrt(np.mean(s**2)))),
            sMAPE=("pct_err","mean")
        ).reset_index().sort_values("MAE", ascending=True)
        print("\nResumen por grupo (ordenado por MAE ascendente):")
        display(agg.head(20))
        df_errors_rank = agg
else:
    print("No hay predicciones para analizar errores.")


## 4) Resumen Ejecutivo + Exportables

In [ ]:

print_header("RESUMEN EJECUTIVO DE EVALUACIÓN")

ev = {
    "target": TARGET,
    "metrics": {},
    "best_artifact": None,
    "generated_at": datetime.now().isoformat(timespec="seconds"),
}

# Seleccionar mejor artefacto (heurística por nombre)
arts = sorted(glob.glob(str(DIR_MODELS / f"*{TARGET}*.joblib")))
ev["best_artifact"] = arts[-1] if arts else "desconocido"

# Métricas del modelo (si existen)
if metrics_model:
    ev["metrics"].update(metrics_model)

# Si existen baselines, agregar para comparación
if baseline_metrics:
    ev["metrics_baselines"] = baseline_metrics

# Guardar JSON
out_json = DIR_RES / f"ev_{TARGET}.json"
out_json.write_text(json.dumps(ev, ensure_ascii=False, indent=2), encoding="utf-8")
print("💾 Guardado:", out_json)

# Mostrar resumen legible
if metrics_model:
    print("\nModelo (predicciones):\n" + format_metrics(metrics_model))
if baseline_metrics:
    for name, met in baseline_metrics.items():
        print(f"\nBaseline — {name}:\n" + format_metrics(met))

# Guardar tablas auxiliares si existen
if 'df_baselines_view' in globals() and df_baselines_view is not None:
    p = DIR_RES / f"baseline_view_{TARGET}.csv"
    df_baselines_view.to_csv(p, index=False)
    print("💾 Guardado:", p)

if 'df_eval' in globals() and df_eval is not None and not df_eval.empty:
    tmp = df_eval.copy()
    if "abs_err" not in tmp.columns:
        tmp["abs_err"] = (tmp[y_true_col] - tmp[y_pred_col]).abs()
    p = DIR_RES / f"eval_rows_{TARGET}.csv"
    tmp.to_csv(p, index=False)
    print("💾 Guardado:", p)

if 'df_errors_rank' in globals() and df_errors_rank is not None:
    p = DIR_RES / f"errors_by_group_{TARGET}.csv"
    df_errors_rank.to_csv(p, index=False)
    print("💾 Guardado:", p)

print("\n✅ Listo. Este notebook está preparado para ejecutarse aun si faltan piezas; imprime advertencias claras.")


## 5) (Opcional) Evaluar **todos** los targets en lote

In [ ]:

# Puedes ejecutar esta celda para correr 3 evaluaciones rápidas y generar 3 JSONs en /results
for tgt in ["price","consumption","profit"]:
    TARGET = tgt
    HOLDOUT_YEAR = None
    print_header(f"[Lote] Evaluando target={tgt}")
    # Para simplificar, re-ejecuta desde las celdas 5→10 manualmente si lo deseas.
    print("Sugerencia: cambia arriba TARGET y vuelve a correr las celdas 5→10 en orden.")
print("Fin del modo lote.")
